In [28]:
import numpy as np
import random
import pandas as pd 
from copy import deepcopy
import math
import time
from math import ceil,floor

In [29]:
def g1(x):
    x1=x[0]
    x2=x[1]
    x3=x[2]
    x4=x[3]
    
    return (-x1 + 0.0193*x3)

In [30]:
def g2(x):
    x1=x[0]
    x2=x[1]
    x3=x[2]
    x4=x[3]
   
    
    return (-x2 + 0.0095*x3)

In [31]:
def g3(x):
    x1=x[0]
    x2=x[1]
    x3=x[2]
    x4=x[3]
    pi=math.pi

    return  (-pi*x3**2*x4 - (4/3)*pi*x3**3 + 1296000) 

In [32]:
def g4(x):
    x1=x[0]
    x2=x[1]
    x3=x[2]
    x4=x[3]
    return (x4-240)

In [33]:
def is_feasable(x):
    x1=x[0]
    x2=x[1]
    x3=x[2]
    x4=x[3]
  
    if x1<0.0625  or x1>99*0.0625:
        #print("x1")
        return False
    if x2<0.0625 or x2>99*0.0625:
        #print("x2")
        return False
        
    if x3<10 or x3>200:
        #print("x3")
        return False
    if x4<10 or x4>200:
        #print("x4")
        return False
    

    g1res=g1(x)
    g2res=g2(x)
    g3res=g3(x)
    g4res=g4(x)
    
    if  g1res>0:
        #print("g1")
        return False
    if g2res>0:
        #print("g2")
        return False
    if  g3res>0:
        #print("g3")
        return False
    if g4res>0:
        return False
    return True
    
    

In [34]:
def vessel(x):
    x1=x[0]
    x2=x[1]
    x3=x[2]
    x4=x[3]
    
    toReturn= -(0.6224*x1*x3*x4 + 1.7781*x2*x3**2 + 3.1661*x1**2*x4 + 19.84*x1**2*x3)
    penalty=50000
    g1res=g1(x)
    if g1res>0:
        toReturn+=-g1res*penalty
    
    
    g2res=g2(x)
    if g2res>0:
        toReturn+=-g2res*penalty
    
    
    g3res=g3(x)
    if g3res>0:
        toReturn+=-g3res*penalty
    g4res=g4(x)
    if g4res>0:
        toReturn+=-g4res*penalty
    return toReturn

In [35]:
def actual_vessel(x):
    x1=x[0]
    x2=x[1]
    x3=x[2]
    x4=x[3]
    
    
    return -(0.6224*x1*x3*x4 + 1.7781*x2*x3**2 + 3.1661*x1**2*x4 + 19.84*x1**2*x3)

In [36]:
class Individual:
   
    def __init__(self,bounds,calc_fitness):
        self.lower_bounds = np.array([bound[0] for bound in bounds])
        self.upper_bounds = np.array([bound[1] for bound in bounds])        
        
        self.position = np.random.uniform(self.lower_bounds, self.upper_bounds, len(bounds))
        
        self.calc_fitness=calc_fitness
        self.fitness=calc_fitness(self.position)
        
    def clip(self):
        self.position = np.clip(self.position ,
                                self.lower_bounds,
                                self.upper_bounds)
        self.update_fitness()
    def update_fitness(self):
        self.fitness=self.calc_fitness(self.position)
    def mutate(self):
        for i in range(len(self.position)):
            self.position+=(-1)**(random.randrange(1,3))*np.random.rand(4)
        self.clip()

In [37]:
def selection(population:Individual) ->Individual:
    #roulete
    ranks= list.reverse(list(range(1,len(population)+1)))
    return random.choices(population=list(zip(range(len(population)),population)),weights=ranks,k=1)[0]

In [38]:
def crossover(p1:Individual,p2:Individual,c1:Individual,c2:Individual):
    coef=random.random()
    c1.position=coef*p1.position + (1-coef)*p2.position

    c2.position=coef*p2.position +(1-coef)*p1.position

In [39]:
def mutation(ind:Individual,mut_prob=0.01):
    if random.random()<mut_prob:
        ind.mutate()
    pass

In [40]:
def GA(pop_size,allowed_time,elitism_size=6):
    bounds = [(0.0625,99* 0.0625),(0.0625,99* 0.0625),(10,200),(10,200)]
    population=[Individual(bounds,vessel) for _ in range(pop_size)]
    
    new_population=population[:]
    start=time.time()
    while True:
        sorted(population,key=lambda x:x.fitness,reverse=True)
        for i in range(elitism_size,pop_size,2):
            p1=selection(population)[1]
            p2=selection(population)[1]
            
            crossover(p1,p2,new_population[i],new_population[i+1])
            mutation(new_population[i])
            mutation(new_population[i+1])
            new_population[i].update_fitness()
            new_population[i+1].update_fitness()
        population=new_population[:]
        if time.time()-start>allowed_time:
            tmp=max(population,key=lambda x:x.fitness)
            
            return tmp
    

In [41]:
#x=GA(50,0.1)
#print(actual_himmelblau(x.position))
#print(is_feasable(x.position))

times=[10**(-3),10**(-2),10**(-1),1]
num_trys=1000
print("Function : Himmelblau")
print("Population size: " + "50")
print("Num dimensions: " + "5")


for allowed_time in times:
    suma=0
    counter=0
    for _ in range(num_trys):
        ind=GA(50,allowed_time)
        if is_feasable(ind.position):
            suma+=actual_vessel(ind.position)
            counter+=1
        
    suma=-suma
    suma/=counter
    
    print("    Time : " + str(allowed_time))
    print("    NumHits : " + str(counter) + "/" + str(num_trys))
    print("    Average minimum: " +str(suma))
    print("\n")

Function : Himmelblau
Population size: 50
Num dimensions: 5
    Time : 0.001
    NumHits : 826/1000
    Average minimum: 47278.779726607216


    Time : 0.01
    NumHits : 799/1000
    Average minimum: 48278.058944957746


    Time : 0.1
    NumHits : 805/1000
    Average minimum: 47913.48869576948


    Time : 1
    NumHits : 790/1000
    Average minimum: 46872.178453021836


